# Transformer






In [5]:
import torch
checkpoints_path = "/root/eigenestimation/outputs/eigenmodels/"

eigenmodel_path = checkpoints_path + "tinystories-8M-eigenmodel.pt"
eigenmodel = torch.load(eigenmodel_path)['model']
from eigenestimation.evaluation.top_logits import compute_jacobian
from importlib import reload
# reload compute jacobian
import sys


In [6]:
import sys
sys.path.append('/root/eigenestimation')  # Ensure the module path is in sys.path

import eigenestimation.evaluation.top_logits  # Import the full module
from eigenestimation.evaluation.top_logits import compute_jacobian  # Import the function

from importlib import reload
reload(eigenestimation.evaluation.top_logits)  # Reload the entire module

# If you need to ensure the function is updated, re-import it
from eigenestimation.evaluation.top_logits import compute_jacobian


In [13]:
X_data_path = checkpoints_path + "tinystories-8M-X_data.pt"
attributions_path = checkpoints_path + "tinystories-8M-circuit_attributions.pt"
eigenmodel_path = checkpoints_path + "tinystories-8M-eigenmodel.pt"
X_data = torch.load(X_data_path)
attributions = torch.load(attributions_path)
eigenmodel = torch.load(eigenmodel_path)['model']
tokenizer = eigenmodel.model.tokenizer
frac_activated = torch.load(eigenmodel_path)['frac_activated']

In [ ]:
circuit_idxs = range(len(attributions))
circuit_idxs = range(100)
for circuit_idx in circuit_idxs:
    print('\n\n----- circuit_idx:', circuit_idx, 'activation:', attributions[circuit_idx]['activation'], '-----')
    for example in attributions[circuit_idx]['top_examples']:

            
        
        tokens = (X_data[example['sample_id']][:example['token_id']+1])
        text = tokenizer.decode(tokens.long())
        
        jac = (compute_jacobian(eigenmodel, tokens.to('cuda'), circuit_idx, device='cuda'))
        top_token_idxs = (jac.argsort(descending=True)[:5])
        top_logits = [eigenmodel.model.tokenizer.decode(token_idx) for token_idx in top_token_idxs]
        bottom_token_idxs = jac.argsort(descending=False)[:5]
        bottom_logits = [eigenmodel.model.tokenizer.decode(token_idx) for token_idx in bottom_token_idxs]
        
        print(text.replace('\n', '\\newline'), '-->', example['value'], 'top_logits', top_logits, 'bottom_logits', bottom_logits)




----- circuit_idx: 0 activation: 0.059 -----
 to play in --> 59.854 top_logits [' experiments', ' coughing', ' register', ' video', ' studies'] bottom_logits [' 94', ' Lois', 'gyn', ' Comed', ' watered']
 to play games with his friends in --> 56.125 top_logits [' register', ' password', ' methods', ' questions', ' stereo'] bottom_logits ['otive', 'Arg', ' bananas', 'gae', ' prepar']
 the mineral and decided to keep it in --> 54.804 top_logits [' cooking', 'ked', ' their', ' curry', ' overnight'] bottom_logits [' facets', ' wiping', 'stable', ' trait', 'ape']
 that the judge was hoping to find a good home for all the dogs in --> 54.741 top_logits [' arises', 'Els', ' Paulo', ' Diana', ' Drew'] bottom_logits [' flat', ' Roma', 'nit', ' xen', 'Detect']
 to play with his friends in --> 54.335 top_logits [' register', ' password', ' speakers', ' stereo', ' recorder'] bottom_logits [' Mas', 'edIn', ' Youth', ' ric', ' eleven']


----- circuit_idx: 1 activation: 0.034 -----
 She did not see

In [40]:
circuit_idxs = range(len(attributions))
circuit_idxs = [69, 4, 3, 2, 5, 7, 9, 11, 16, 17, 18, 24, 26, 29, 31, 33, 34, 44, 46, 47, 51, 54, 62, 63, 74, 79, 86, 91, 98, 99]


def highlight_last_word(text):
    text = text.replace('\n', '\\newline')  # Replace newlines
    words = text.split()  # Split into words

    if len(words) > 1:
        highlighted_text = " ".join(words[:-1]) + f" \033[1;31m{words[-1]}\033[0m"  # Red + Bold last word
    else:
        highlighted_text = f"\033[1;31m{text}\033[0m"  # Just highlight if it's a single word

    print(highlighted_text)

# Example Usage
highlight_last_word("This is an example sentence.")
highlight_last_word("Hello\nworld")

for circuit_idx in circuit_idxs:
    print('\n\n----- circuit_idx:', circuit_idx, 'activation:', attributions[circuit_idx]['activation'], '-----')
    for example in attributions[circuit_idx]['top_examples']:
        i = example['token_id']
        tokens = (X_data[example['sample_id']])
        token_list = tokenizer.convert_ids_to_tokens(tokens.long())  # Convert tokens to words
        text_tokens = token_list[:]  # Copy list for safe modification
        
        # Highlight the i-th token
        if 0 <= i < len(text_tokens):
            text_tokens[i] = f"\033[1;31m{text_tokens[i]}\033[0m"  # Red and bold in terminal
        
        text = "".join(text_tokens[:(i+1)])  # Reconstruct the highlighted text
        
        jac = (compute_jacobian(eigenmodel, tokens.to('cuda'), circuit_idx, device='cuda'))
        top_token_idxs = (jac.argsort(descending=True)[:3])
        top_logits = [eigenmodel.model.tokenizer.decode(token_idx) for token_idx in top_token_idxs]
        bottom_token_idxs = jac.argsort(descending=False)[:3]
        bottom_logits = [eigenmodel.model.tokenizer.decode(token_idx) for token_idx in bottom_token_idxs]
        
        # Print with highlighted i-th token
        print(text.replace('\n', '\\newline').replace('Ġ', ' ').replace('Ċ', ' '))  # Highlighted text


This is an example sentence.
Hello\newlineworld


----- circuit_idx: 69 activation: 0.064 -----
 and Max became good friends, and they played together every day.Once upon
 them disappear again.Once upon
 dad very much.Once upon
, Monkey always kept his room tidy just like Ellie's.Once upon
 was more important than being bossy.Once upon


----- circuit_idx: 4 activation: 0.109 -----
 came up to him and asked, "What are
 very happy. Toot asked, "Why are
 in the car and drove to the vet. They did
  Benny asked, "What are
 for a walk and they passed by


----- circuit_idx: 3 activation: 0.204 -----
 with her mom. One
 the park with her friends. One
!" she said.  One
 and they pretended it was a treasure.  One
 just made a loud noise. Timmy was confused and sad.  One


----- circuit_idx: 2 activation: 0.185 -----
 teddy bear and had a lot of
 They see the box and the paper. They are
 a bench to sit and eat their ice cream. They l
-cream, and had lots of
.Lily likes to play with her dolls. S

In [34]:
token_list

['Ġin', 'Ġthe', 'Ġpark', '.', 'Once', 'Ġupon', 'Ġa', 'Ġtime', ',', 'Ġthere', 'Ġwas', 'Ġa', 'Ġlittle', 'Ġgirl', 'Ġnamed', 'ĠLily']

In [32]:
text

'Ġin Ġthe Ġpark . Once Ġupon Ġa Ġtime , Ġthere Ġwas Ġa Ġlittle Ġgirl \x1bĠnamed\x1b ĠLily'

In [25]:
pprint

Pretty printing has been turned OFF


In [17]:
tokens

tensor([  51,  258,  582, 8712,   11,  366,   40,  836])